In [27]:
import numpy as np
import pandas as pd
import re
from unidecode import unidecode
from nltk.corpus import stopwords
from nltk.tokenize import wordpunct_tokenize
from nltk.stem.lancaster import LancasterStemmer

In [28]:
# import nltk
# nltk.download('stopwords')

In [29]:
matrix = pd.read_parquet('../datasets/processed/frequency_matrix_2013.parquet')

In [30]:
def remove_stopwords(text: str) -> str:
    stop_words = set(stopwords.words('portuguese'))
    new_text = [item for item in wordpunct_tokenize(text) if item not in stop_words]
    return " ".join(new_text)

In [31]:
def lc_stem(text: str) -> str:
    stemmer = LancasterStemmer()
    words = [stemmer.stem(word) for word in wordpunct_tokenize(text)]
    return " ".join(words)

In [32]:
resumo = 'Este trabalho foi desenvolvido utilizando as tecnologias Python, Pandas, Numpy, NLTK, Unidecode e LancasterStemmer. O objetivo é realizar a limpeza de um texto, removendo stopwords, pontuações e acentuações, além de realizar a redução de palavras para sua forma raiz. O texto utilizado para teste foi o resumo deste trabalho. nitrosilo complexo rutenio'

In [33]:
resumo = resumo.lower()

In [34]:
resumo = re.sub(r'[^a-zA-Z0-9\s]+', '', resumo)

In [35]:
resumo = remove_stopwords(resumo)
resumo = lc_stem(resumo)
resumo = unidecode(resumo)

del remove_stopwords, lc_stem

In [36]:
resumo

'trabalho desenvolvido utilizando tecnologia python panda numpy nltk unidecod lancasterstem objetivo realiz limpez texto removendo stopword pontua acentua alm realiz reduo palavra form raiz texto utilizado test resumo dest trabalho nitrosilo complexo rutenio'

In [37]:
def verify_concat(i: int, item: str, text: str) -> str:
    elements = wordpunct_tokenize(text)[i:]
    
    if len(elements) > 1:
        test_1 = item + ' ' + elements[1]
        test_2 = item  + elements[1]
    else:
        return i, item
    
    if test_1 in matrix.palavra_chave.values:
        return verify_concat(i+1, test_1, text)
    if test_2 in matrix.palavra_chave.values:
        return verify_concat(i+1, test_2, text)
    else:
        return i, item

def get_keywords(text: str) -> list:
    result = []
    i = 0
    elements = wordpunct_tokenize(text)
    while i < len(elements):
        item = elements[i]
        if item in matrix.palavra_chave.values:
            i, item = verify_concat(i, item, text)
            result.append(item)
        i+=1
    return result

In [39]:
result = get_keywords(resumo)
result

['trabalho',
 'utilizando',
 'tecnologia',
 'python',
 'nltk',
 'objetivo',
 'limpez',
 'texto',
 'alm',
 'palavra',
 'form',
 'raiz',
 'texto',
 'test',
 'resumo',
 'trabalho',
 'nitrosilo complexo rutenio']